In [1]:
!pwd

/content


In [1]:
# Declaring the working library
import os
os.chdir('/content')

In [2]:
# Installing the required library
# !pip install -U scikit-learn

In [3]:
import sklearn
print(sklearn.__version__)

1.5.1


__Evaluation Module__

In [4]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [5]:
# import numpy as np
from sklearn.metrics import mean_squared_log_error, r2_score

EPSILON = 1e-10


def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted


def _percentage_error(actual: np.ndarray, predicted: np.ndarray):
    """
    Percentage error
    Note: result is NOT multiplied by 100
    """
    return _error(actual, predicted) / (actual + EPSILON)


def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    """ Naive forecasting method which just repeats previous samples """
    return actual[:-seasonality]


def _relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark
        return _error(actual[seasonality:], predicted[seasonality:]) /\
               (_error(actual[seasonality:], _naive_forecasting(actual, seasonality)) + EPSILON)

    return _error(actual, predicted) / (_error(actual, benchmark) + EPSILON)


def _bounded_relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Bounded Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark

        abs_err = np.abs(_error(actual[seasonality:], predicted[seasonality:]))
        abs_err_bench = np.abs(_error(actual[seasonality:], _naive_forecasting(actual, seasonality)))
    else:
        abs_err = np.abs(_error(actual, predicted))
        abs_err_bench = np.abs(_error(actual, benchmark))

    return abs_err / (abs_err + abs_err_bench + EPSILON)


def _geometric_mean(a, axis=0, dtype=None):
    """ Geometric mean """
    if not isinstance(a, np.ndarray):  # if not an ndarray object attempt to convert it
        log_a = np.log(np.array(a, dtype=dtype))
    elif dtype:  # Must change the default dtype allowing array type
        if isinstance(a, np.ma.MaskedArray):
            log_a = np.log(np.ma.asarray(a, dtype=dtype))
        else:
            log_a = np.log(np.asarray(a, dtype=dtype))
    else:
        log_a = np.log(a)
    return np.exp(log_a.mean(axis=axis))


def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))


def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))


def nrmse(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Root Mean Squared Error """
    return rmse(actual, predicted) / (actual.max() - actual.min())


def me(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Error """
    return np.mean(_error(actual, predicted))

def r2_scr(actual:np.ndarray, predicted: np.ndarray):
    """Calculating the r2 value: perasonian correlation coefficient"""
    return metrics.r2_score(actual, predicted)



def mae(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))


mad = mae  # Mean Absolute Deviation (it is the same as MAE)


def gmae(actual: np.ndarray, predicted: np.ndarray):
    """ Geometric Mean Absolute Error """
    return _geometric_mean(np.abs(_error(actual, predicted)))


def mdae(actual: np.ndarray, predicted: np.ndarray):
    """ Median Absolute Error """
    return np.median(np.abs(_error(actual, predicted)))


def mpe(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Percentage Error """
    return np.mean(_percentage_error(actual, predicted))


def mape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Absolute Percentage Error
    Properties:
        + Easy to interpret
        + Scale independent
        - Biased, not symmetric
        - Undefined when actual[t] == 0
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.abs(_percentage_error(actual, predicted)))


def mdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(np.abs(_percentage_error(actual, predicted)))


def smape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Mean Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))


def smdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))


def maape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Arctangent Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))


def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))


def std_ae(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Error """
    __mae = mae(actual, predicted)
    return np.sqrt(np.sum(np.square(_error(actual, predicted) - __mae))/(len(actual) - 1))


def std_ape(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Percentage Error """
    __mape = mape(actual, predicted)
    return np.sqrt(np.sum(np.square(_percentage_error(actual, predicted) - __mape))/(len(actual) - 1))


def rmspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Mean Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.mean(np.square(_percentage_error(actual, predicted))))


def rmdspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Median Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.median(np.square(_percentage_error(actual, predicted))))


def rmsse(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """ Root Mean Squared Scaled Error """
    q = np.abs(_error(actual, predicted)) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))
    return np.sqrt(np.mean(np.square(q)))


def inrse(actual: np.ndarray, predicted: np.ndarray):
    """ Integral Normalized Root Squared Error """
    return np.sqrt(np.sum(np.square(_error(actual, predicted))) / np.sum(np.square(actual - np.mean(actual))))


def rrse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Relative Squared Error """
    return np.sqrt(np.sum(np.square(actual - predicted)) / np.sum(np.square(actual - np.mean(actual))))


def mre(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Error """
    return np.mean(_relative_error(actual, predicted, benchmark))


def rae(actual: np.ndarray, predicted: np.ndarray):
    """ Relative Absolute Error (aka Approximation Error) """
    return np.sum(np.abs(actual - predicted)) / (np.sum(np.abs(actual - np.mean(actual))) + EPSILON)


def mrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Absolute Error """
    return np.mean(np.abs(_relative_error(actual, predicted, benchmark)))


def mdrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Median Relative Absolute Error """
    return np.median(np.abs(_relative_error(actual, predicted, benchmark)))


def gmrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Geometric Mean Relative Absolute Error """
    return _geometric_mean(np.abs(_relative_error(actual, predicted, benchmark)))


def mbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Bounded Relative Absolute Error """
    return np.mean(_bounded_relative_error(actual, predicted, benchmark))


def umbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Unscaled Mean Bounded Relative Absolute Error """
    __mbrae = mbrae(actual, predicted, benchmark)
    return __mbrae / (1 - __mbrae)


def mda(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Directional Accuracy """
    return np.mean((np.sign(actual[1:] - actual[:-1]) == np.sign(predicted[1:] - predicted[:-1])).astype(int))

# https://www.kaggle.com/carlolepelaars/understanding-the-metric-rmsle
def rmsle(y_true:np.ndarray, y_pred:np.ndarray):
    """
        The Root Mean Squared Log Error (RMSLE) metric

        :param y_true: The ground truth labels given in the dataset/actual values
        :param y_pred: Our predictions/forecasts
        :return: The RMSLE score --> Evaluation metric
    """
    return np.sqrt(mean_squared_log_error(y_true, y_pred))


METRICS = {
    'mse': mse,
    'rmse': rmse,
    'nrmse': nrmse,
    'me': me,
    'mae': mae,
    'mad': mad,
    'gmae': gmae,
    'mdae': mdae,
    'mpe': mpe,
    'mape': mape,
    'mdape': mdape,
    'smape': smape,
    'smdape': smdape,
    'maape': maape,
    'mase': mase,
    'std_ae': std_ae,
    'std_ape': std_ape,
    'rmspe': rmspe,
    'rmdspe': rmdspe,
    'rmsse': rmsse,
    'inrse': inrse,
    'rrse': rrse,
    'mre': mre,
    'rae': rae,
    'r2_scr':r2_scr,
    'mrae': mrae,
    'mdrae': mdrae,
    'gmrae': gmrae,
    'mbrae': mbrae,
    'umbrae': umbrae,
    'mda': mda,
    'rmsle': rmsle
}


def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('mae', 'mse', 'smape', 'umbrae')):
    results = {}
    for name in metrics:
        try:
            results[name] = METRICS[name](actual, predicted)
        except Exception as err:
            results[name] = np.nan
            print('Unable to compute metric {0}: {1}'.format(name, err))
    return results


def evaluate_all(actual: np.ndarray, predicted: np.ndarray):
    return evaluate(actual, predicted, metrics=set(METRICS.keys()))

# Module for Theil's U1
def theils_u1(f:np.ndarray , y:np.ndarray):
    df = pd.DataFrame({'f_i':f, 'y_i': y})
    df['(f_i - y_i)^2'] = np.square(df['f_i'] - df['y_i'])
    df['y_i^2'] = np.square(df['y_i'])
    df['f_i^2'] = np.square(df['f_i'])
    return (np.sqrt(np.mean(df['(f_i - y_i)^2'])))/(np.sqrt(np.mean(df['y_i^2']))+np.sqrt(np.mean(df['f_i^2'])))

# Module for Theil's U2
def theils_u2(f:int , y:int)->float:
    y = y.values.flatten()
    f = f.values.flatten()
    df = pd.DataFrame({'f_i+1':f, 'y_i+1': y})
    df['y_i'] = df['y_i+1'].shift(periods=1)
    df['numerator'] = np.square((df['f_i+1'] - df['y_i+1']) / df['y_i'])
    df['denominator'] = np.square((df['y_i+1'] - df['y_i']) / df['y_i'])
    df.dropna(inplace=True)
    return np.sqrt(np.sum(df['numerator'])/np.sum(df['denominator']))

For the forecast values across different algorithms , please refer to 'Table_05_06_Forecasts_FEWNet_Baselines_12M_24M_BRIC_Comparison.xlsx' at "/FEWNet/dataset"

__Brazil: Forecasts Evaluation:  FEWNet and baselines:12M (table 05)__

In [6]:
actual = np.array([4.517457  ,4.559198  ,5.195379  ,6.099479  ,6.759116  ,8.056819  ,
                   8.347072  ,8.994823  ,9.679774 ,10.246209 ,10.672622 ,10.738501])
fewnet = np.array([4.51,
4.97,
5.53,
5.86,
6.29,
7.02,
7.62,
8.06,
8.33,
8.40,
8.30,
8.21])

warimax = np.array([5.384066,4.751891,4.339061,4.378773,4.524725,4.851346,5.137784,5.464164,5.688167,5.934548,6.102688,6.309670])
rwd = np.array([4.294411 ,4.277599 ,4.260786 ,4.243974 ,4.227162 ,4.210349 ,4.193537 ,4.176725 ,4.159912 ,4.143100 ,4.126287 ,4.109475])
ar = np.array([4.458540 ,
4.513750 ,
4.548190 ,
4.582458 ,
4.618596 ,
4.654877 ,
4.690014 ,
4.723529 ,
4.755369 ,
4.785614 ,
4.814358 ,
4.841688])
msgarch = np.array([4.24,
4.68,
4.77,
4.87,
4.96,
5.31,
4.79,
5.52,
4.61,
5.26,
5.13,
4.72])
rw = np.array([4.315522,
4.319820,
4.324119,
4.328418,
4.332716,
4.337015,
4.341313,
4.345612,
4.349910,
4.354209,
4.358507,
4.362806])

deepar = np.array([9.94,
10.03,
10.05,
10.09,
9.84,
9.28,
8.84,
8.44,
8.03,
7.85,
7.66,
7.35])

arnnx = np.array([9.02,
7.78,
6.53,
6.01,
6.32,
6.83,
7.71,
7.41,
7.34,
7.48,
8.24,
8.80])

nbeats = np.array([4.11,
4.34,
4.46,
5.07,
5.05,
5.28,
5.52,
5.97,
6.11,
6.16,
6.09,
5.77])
arfimax = np.array([6.81,
5.18,
3.95,
3.24,
3.04,
3.19,
3.45,
3.72,
3.94,
4.09,
4.15,
4.14])
sarimax = np.array([7.81,
6.35,
5.59,
5.51,
5.78,
5.93,
6.65,
6.90,
7.05,
7.25,
7.23,
7.21])
arimax = np.array([7.87,
6.29,
5.43,
5.21,
5.40,
5.76,
6.13,
6.43,
6.64,
6.73,
6.74,
6.73])
lr = 	np.array([6.56,
8.39,
7.55,
7.01,
6.93,
7.05,
7.20,
7.43,
7.62,
7.74,
7.80,
7.83])
ardl = np.array([3.39,
2.42,
2.46,
3.14,
3.86,
4.54,
5.21,
5.74,
6.09,
6.31,
6.44,
6.49])
xgboost = np.array([4.73,
5.11,
5.04,
4.82,
4.88,
5.85,
6.35,
6.75,
6.66,
6.57,
6.81,
7.25])
tft = 	np.array([3.20,
4.52,
2.34,
2.03,
3.15,
2.36,
4.00,
3.00,
1.90,
3.83,
2.94,
2.82])

In [7]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results


print("Evaluation_metrics:  rw - ", evaluate(actual,rw))
print("Measuring Theils U1: rw - ", theils_u1(actual,rw))

print("Evaluation_metrics:  rwd - ", evaluate(actual,rwd))
print("Measuring Theils U1: rwd - ", theils_u1(actual,rwd))

print("Evaluation_metrics:  ar - ", evaluate(actual,ar))
print("Measuring Theils U1: ar - ", theils_u1(actual,ar))

print("Evaluation_metrics:  msgarch - ", evaluate(actual,msgarch))
print("Measuring Theils U1: msgarch - ", theils_u1(actual,msgarch))

print("Evaluation_metrics:  deepar - ", evaluate(actual,deepar))
print("Measuring Theils U1: deepar - ", theils_u1(actual,deepar))

print("Evaluation_metrics:  arnnx - ", evaluate(actual,arnnx))
print("Measuring Theils U1: arnnx - ", theils_u1(actual,arnnx))

print("Evaluation_metrics:  nbeats - ", evaluate(actual,nbeats))
print("Measuring Theils U1: nbeats - ", theils_u1(actual,nbeats))

print("Evaluation_metrics:  arfimax - ", evaluate(actual,arfimax))
print("Measuring Theils U1: arfimax - ", theils_u1(actual,arfimax))

print("Evaluation_metrics:  sarimax - ", evaluate(actual,sarimax))
print("Measuring Theils U1: sarimax - ", theils_u1(actual,sarimax))

print("Evaluation_metrics:  arimax - ", evaluate(actual,arimax))
print("Measuring Theils U1: arimax - ", theils_u1(actual,arimax))

print("Evaluation_metrics:  lr - ", evaluate(actual,lr))
print("Measuring Theils U1: lr - ", theils_u1(actual,lr))

print("Evaluation_metrics:  ardl - ", evaluate(actual,ardl))
print("Measuring Theils U1: ardl - ", theils_u1(actual,ardl))

print("Evaluation_metrics:  xgboost - ", evaluate(actual,xgboost))
print("Measuring Theils U1: xgboost - ", theils_u1(actual,xgboost))

print("Evaluation_metrics:  tft - ", evaluate(actual,tft))
print("Measuring Theils U1: tft - ", theils_u1(actual,tft))

print("Evaluation_metrics:  warimax - ", evaluate(actual,warimax))
print("Measuring Theils U1: warimax - ", theils_u1(actual,warimax))

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

Evaluation_metrics:  rw -  {'rmse': 4.133401612636921, 'mase': 6.158683628235606, 'smape': 0.5199990480502511, 'mdrae': 7.17746634012492, 'mdape': 0.47079812857046627}
Measuring Theils U1: rw -  0.33131115510414716
Evaluation_metrics:  rwd -  {'rmse': 4.287897288243019, 'mase': 6.4013164028417116, 'smape': 0.5480375831336378, 'mdrae': 7.438194613757731, 'mdape': 0.4875108895528768}
Measuring Theils U1: rwd -  0.34750641919697445
Evaluation_metrics:  ar -  {'rmse': 3.804146060002976, 'mase': 5.581513944176145, 'smape': 0.45479952860361345, 'mdrae': 6.594036905138856, 'mdape': 0.4301842331215173}
Measuring Theils U1: ar -  0.29711099642382705
Evaluation_metrics:  msgarch -  {'rmse': 3.591668971783951, 'mase': 5.193777001094995, 'smape': 0.41418716439376185, 'mdrae': 5.364442508716404, 'mdape': 0.3636223080715831}
Measuring Theils U1: msgarch -  0.27515380245084264
Evaluation_metrics:  deepar -  {'rmse': 3.4081455946808155, 'mase': 5.236413413890016, 'smape': 0.368145517107865, 'mdrae': 4

__Brazil: Forecasts Evaluation: FEWNet and baselines:24M (table 06)__

In [8]:
actual = np.array([4.306152,4.191771,4.005114,3.303158,2.399279,1.877727,2.132417,2.305625,2.438465,3.135329,
3.918350,4.311223,4.517457,4.559198,5.195379,6.099479,6.759116,8.056819,8.347072,8.994823
,9.679774,10.246209,10.672622,10.738501])
rw = np.array([3.28,
3.29,
3.29,
3.30,
3.31,
3.31,
3.32,
3.33,
3.34,
3.34,
3.35,
3.36,
3.36,
3.37,
3.38,
3.38,
3.39,
3.40,
3.40,
3.41,
3.42,
3.42,
3.43,
3.44])
rwd = np.array([3.25,
3.23,
3.20,
3.18,
3.16,
3.13,
3.11,
3.09,
3.06,
3.04,
3.02,
2.99,
2.97,
2.95,
2.92,
2.90,
2.88,
2.85,
2.83,
2.81,
2.79,
2.76,
2.74,
2.72])
ar = np.array([3.71,
3.99,
4.17,
4.30,
4.40,
4.49,
4.56,
4.62,
4.68,
4.74,
4.79,
4.83,
4.88,
4.92,
4.95,
4.99,
5.02,
5.05,
5.08,
5.11,
5.13,
5.16,
5.18,
5.20])
msgarch = np.array([3.50,
3.93,
4.31,
4.82,
5.08,
5.49,
5.08,
5.14,
5.02,
4.93,
4.69,
4.83,
5.07,
5.37,
5.83,
5.39,
5.06,
5.09,
4.91,
5.16,
5.52,
5.41,
5.59,
5.49])
deepar = np.array([10.03,
10.57,
10.71,
10.68,
10.70,
10.22,
9.84,
9.58,
8.96,
8.24,
7.79,
7.31,
7.13,
6.70,
5.91,
5.06,
4.26,
3.72,
3.28,
2.80,
2.35,
2.03,
1.63,
1.34])
arnnx = np.array([10.11,
6.98,
6.41,
6.32,
6.25,
6.08,
7.04,
6.65,
6.49,
7.37,
7.85,
8.38,
5.54,
7.31,
7.01,
7.75,
7.85,
7.73,
6.37,
6.09,
6.09,
5.55,
5.99,
6.20])
nbeats = np.array([2.90,
3.53,
3.92,
4.04,
4.57,
4.80,
4.88,
4.50,
4.68,
4.99,
5.04,
5.27,
5.00,
5.35,
4.88,
4.50,
4.56,
3.98,
4.49,
4.44,
4.26,
4.60,
4.73,
4.70])
arfimax = np.array([5.51,
3.77,
2.54,
1.95,
1.74,
1.89,
2.18,
2.46,
2.68,
2.86,
2.94,
2.95,
2.95,
3.01,
3.16,
3.33,
3.38,
3.20,
2.78,
2.27,
1.85,
1.64,
1.62,
1.65])
sarimax = np.array([8.24,
8.27,
8.47,
8.65,
8.37,
7.76,
7.39,
6.86,
6.43,
6.58,
6.73,
6.61,
6.44,
6.09,
5.78,
5.56,
5.40,
5.03,
5.26,
5.04,
4.80,
4.82,
4.74,
4.64])
arimax = np.array([8.38,
8.55,
8.73,
8.77,
8.53,
8.00,
7.36,
6.82,
6.54,
6.49,
6.51,
6.42,
6.16,
5.75,
5.33,
4.99,
4.73,
4.52,
4.33,
4.15,
3.99,
3.85,
3.74,
3.64])
lr = 	np.array([5.40,
8.06,
7.67,
6.97,
6.77,
6.91,
7.17,
7.39,
7.57,
7.64,
7.66,
7.64,
7.70,
7.76,
7.87,
8.04,
8.11,
7.79,
7.28,
6.69,
6.23,
6.00,
6.01,
6.02])
ardl = np.array([3.51,
2.77,
2.61,
3.39,
4.23,
4.82,
5.44,
6.05,
6.44,
6.64,
6.80,
6.95,
7.09,
7.30,
7.59,
7.78,
7.67,
7.24,
6.64,
6.15,
5.93,
5.96,
6.06,
6.02])
xgboost = np.array([4.48,
4.52,
4.33,
4.08,
4.08,
4.54,
5.15,
5.74,
5.93,
5.94,
6.11,
6.46,
6.56,
6.62,
6.69,
6.95,
7.11,
7.01,
6.63,
6.25,
6.06,
6.10,
6.17,
6.00])
tft = 	np.array([3.63,
5.12,
4.90,
4.96,
5.86,
5.45,
6.57,
6.13,
5.59,
6.82,
6.55,
6.45,
6.44,
6.41,
5.38,
5.91,
5.45,
5.29,
5.31,
4.45,
5.09,
4.03,
3.52,
3.63])
warimax = np.array([5.27,
4.34,
3.57,
3.04,
2.79,
2.92,
2.96,
2.70,
2.69,
2.93,
2.92,
2.79,
2.79,
2.80,
2.80,
2.90,
2.99,
2.87,
2.63,
2.49,
2.40,
2.30,
2.25,
2.21])
fewnet = np.array([4.62,
5.45,
5.63,
5.26,
5.02,
5.08,
5.21,
5.20,
5.10,
5.31,
5.91,
6.04,
6.07,
5.98,
6.05,
6.33,
6.75,
6.90,
6.94,
6.97,
6.96,
6.87,
6.58,
6.55])

In [9]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results


print("Evaluation_metrics:  rw - ", evaluate(actual,rw))
print("Measuring Theils U1: rw - ", theils_u1(actual,rw))

print("Evaluation_metrics:  rwd - ", evaluate(actual,rwd))
print("Measuring Theils U1: rwd - ", theils_u1(actual,rwd))

print("Evaluation_metrics:  ar - ", evaluate(actual,ar))
print("Measuring Theils U1: ar - ", theils_u1(actual,ar))

print("Evaluation_metrics:  msgarch - ", evaluate(actual,msgarch))
print("Measuring Theils U1: msgarch - ", theils_u1(actual,msgarch))

print("Evaluation_metrics:  deepar - ", evaluate(actual,deepar))
print("Measuring Theils U1: deepar - ", theils_u1(actual,deepar))

print("Evaluation_metrics:  arnnx - ", evaluate(actual,arnnx))
print("Measuring Theils U1: arnnx - ", theils_u1(actual,arnnx))

print("Evaluation_metrics:  nbeats - ", evaluate(actual,nbeats))
print("Measuring Theils U1: nbeats - ", theils_u1(actual,nbeats))

print("Evaluation_metrics:  arfimax - ", evaluate(actual,arfimax))
print("Measuring Theils U1: arfimax - ", theils_u1(actual,arfimax))

print("Evaluation_metrics:  sarimax - ", evaluate(actual,sarimax))
print("Measuring Theils U1: sarimax - ", theils_u1(actual,sarimax))

print("Evaluation_metrics:  arimax - ", evaluate(actual,arimax))
print("Measuring Theils U1: arimax - ", theils_u1(actual,arimax))

print("Evaluation_metrics:  lr - ", evaluate(actual,lr))
print("Measuring Theils U1: lr - ", theils_u1(actual,lr))

print("Evaluation_metrics:  ardl - ", evaluate(actual,ardl))
print("Measuring Theils U1: ardl - ", theils_u1(actual,ardl))

print("Evaluation_metrics:  xgboost - ", evaluate(actual,xgboost))
print("Measuring Theils U1: xgboost - ", theils_u1(actual,xgboost))

print("Evaluation_metrics:  tft - ", evaluate(actual,tft))
print("Measuring Theils U1: tft - ", theils_u1(actual,tft))

print("Evaluation_metrics:  warimax - ", evaluate(actual,warimax))
print("Measuring Theils U1: warimax - ", theils_u1(actual,warimax))

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

Evaluation_metrics:  rw -  {'rmse': 3.555816399258117, 'mase': 5.338980888576182, 'smape': 0.4991861937985444, 'mdrae': 5.107530353041516, 'mdape': 0.4119374811813378}
Measuring Theils U1: rw -  0.3714659866097715
Evaluation_metrics:  rwd -  {'rmse': 3.933571489676986, 'mase': 5.891666741959873, 'smape': 0.5689430557055086, 'mdrae': 4.528514846584148, 'mdape': 0.39545922825468316}
Measuring Theils U1: rwd -  0.42757604402674076
Evaluation_metrics:  ar -  {'rmse': 2.7303800602784754, 'mase': 4.345859794097585, 'smape': 0.3962671416430164, 'mdrae': 2.636474302891367, 'mdape': 0.3823025564687732}
Measuring Theils U1: ar -  0.24873636776935457
Evaluation_metrics:  msgarch -  {'rmse': 2.7761500228553255, 'mase': 4.635268303062659, 'smape': 0.42406017032076115, 'mdrae': 2.965796307134672, 'mdape': 0.4190532313194846}
Measuring Theils U1: msgarch -  0.24636022404475144
Evaluation_metrics:  deepar -  {'rmse': 6.156445023325795, 'mase': 11.451390886250358, 'smape': 0.9102433545134189, 'mdrae': 

__Russia: Forecasts Evaluation: FEWNet and baselines:12M (table 05)__

In [10]:
actual = np.array([4.912471 ,5.194607 ,5.666425 ,5.782023 ,5.519744 ,6.014352 ,
                   6.511526 ,6.469069 ,6.692744 ,7.408086,8.135334 ,8.403766])

rw = np.array([4.44,
4.45,
4.47,
4.48,
4.49,
4.51,
4.52,
4.53,
4.55,
4.56,
4.58,
4.59])
rwd = np.array([4.39,
4.36,
4.32,
4.29,
4.25,
4.22,
4.19,
4.15,
4.12,
4.08,
4.05,
4.02])
ar = np.array([4.79,
5.07,
5.27,
5.43,
5.57,
5.69,
5.80,
5.91,
6.01,
6.10,
6.19,
6.27])
msgarch = np.array([4.61,
5.18,
5.53,
5.53,
5.84,
6.86,
5.40,
5.69,
5.46,
6.48,
6.58,
5.57])
deepar = np.array([7.48,
7.64,
7.66,
7.67,
7.57,
7.35,
6.95,
6.66,
6.38,
6.02,
5.38,
4.92])
arnnx = np.array([13.00,
11.18,
10.48,
9.69,
8.87,
8.35,
9.20,
10.34,
8.57,
9.71,
10.92,
11.91])
nbeats = np.array([4.43,
4.84,
4.68,
4.46,
4.73,
4.21,
4.30,
4.07,
4.06,
3.83,
3.80,
4.11])
arfimax = np.array([20.36,
19.71,
19.16,
18.75,
18.50,
18.41,
18.47,
18.61,
18.79,
18.95,
19.10,
19.29])
sarimax = np.array([13.76,
13.21,
12.71,
12.46,
12.41,
12.59,
12.63,
12.95,
13.19,
13.16,
13.14,
13.32])
arimax = np.array([14.85,
14.26,
13.80,
13.52,
13.45,
13.59,
13.85,
14.13,
14.34,
14.46,
14.57,
14.78])
lr = 	np.array([5.04,
9.28,
11.66,
11.26,
11.02,
10.94,
10.99,
11.14,
11.32,
11.44,
11.51,
11.57])
ardl = np.array([7.47,
9.13,
8.68,
8.53,
9.35,
10.01,
10.29,
10.63,
11.14,
11.51,
11.56,
11.66])
xgboost = np.array([7.08,
8.94,
11.09,
13.24,
13.47,
13.67,
13.62,
13.45,
13.48,
13.51,
13.53,
13.49])
tft = 	np.array([2.58,
3.34,
2.39,
2.09,
2.72,
2.46,
3.48,
2.51,
2.00,
3.15,
2.63,
2.83])
warimax = np.array([10.60,
10.16,
9.98,
9.86,
9.70,
9.71,
9.79,
9.78,
9.79,
9.83,
9.83,
9.90])
fewnet = np.array([5.44,
6.38,
6.83,
6.78,
6.51,
6.14,
5.84,
5.80,
6.05,
6.50,
7.02,
7.51])

In [11]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results


print("Evaluation_metrics:  rw - ", evaluate(actual,rw))
print("Measuring Theils U1: rw - ", theils_u1(actual,rw))

print("Evaluation_metrics:  rwd - ", evaluate(actual,rwd))
print("Measuring Theils U1: rwd - ", theils_u1(actual,rwd))

print("Evaluation_metrics:  ar - ", evaluate(actual,ar))
print("Measuring Theils U1: ar - ", theils_u1(actual,ar))

print("Evaluation_metrics:  msgarch - ", evaluate(actual,msgarch))
print("Measuring Theils U1: msgarch - ", theils_u1(actual,msgarch))

print("Evaluation_metrics:  deepar - ", evaluate(actual,deepar))
print("Measuring Theils U1: deepar - ", theils_u1(actual,deepar))

print("Evaluation_metrics:  arnnx - ", evaluate(actual,arnnx))
print("Measuring Theils U1: arnnx - ", theils_u1(actual,arnnx))

print("Evaluation_metrics:  nbeats - ", evaluate(actual,nbeats))
print("Measuring Theils U1: nbeats - ", theils_u1(actual,nbeats))

print("Evaluation_metrics:  arfimax - ", evaluate(actual,arfimax))
print("Measuring Theils U1: arfimax - ", theils_u1(actual,arfimax))

print("Evaluation_metrics:  sarimax - ", evaluate(actual,sarimax))
print("Measuring Theils U1: sarimax - ", theils_u1(actual,sarimax))

print("Evaluation_metrics:  arimax - ", evaluate(actual,arimax))
print("Measuring Theils U1: arimax - ", theils_u1(actual,arimax))

print("Evaluation_metrics:  lr - ", evaluate(actual,lr))
print("Measuring Theils U1: lr - ", theils_u1(actual,lr))

print("Evaluation_metrics:  ardl - ", evaluate(actual,ardl))
print("Measuring Theils U1: ardl - ", theils_u1(actual,ardl))

print("Evaluation_metrics:  xgboost - ", evaluate(actual,xgboost))
print("Measuring Theils U1: xgboost - ", theils_u1(actual,xgboost))

print("Evaluation_metrics:  tft - ", evaluate(actual,tft))
print("Measuring Theils U1: tft - ", theils_u1(actual,tft))

print("Evaluation_metrics:  warimax - ", evaluate(actual,warimax))
print("Measuring Theils U1: warimax - ", theils_u1(actual,warimax))

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

Evaluation_metrics:  rw -  {'rmse': 2.1377974220282, 'mase': 5.038521188028161, 'smape': 0.32949187177799866, 'mdrae': 4.005692171351338, 'mdape': 0.274935853397656}
Measuring Theils U1: rw -  0.19442799513468992
Evaluation_metrics:  rwd -  {'rmse': 2.4868081445158605, 'mase': 5.87230829826063, 'smape': 0.3971410853840716, 'mdrae': 4.8411958276649, 'mdape': 0.3274353346337984}
Measuring Theils U1: rwd -  0.2327192645202085
Evaluation_metrics:  ar -  {'rmse': 0.9896407753811901, 'mase': 1.9471408031717006, 'smape': 0.10990040040691755, 'mdrae': 1.8286162420452583, 'mdape': 0.07819110016821623}
Measuring Theils U1: ar -  0.08129630737113414
Evaluation_metrics:  msgarch -  {'rmse': 1.1425208762339603, 'mase': 2.3050871743424266, 'smape': 0.13260851924887504, 'mdrae': 2.1386569640427116, 'mdape': 0.12285499082217655}
Measuring Theils U1: msgarch -  0.09333967069773833
Evaluation_metrics:  deepar -  {'rmse': 2.0016712269910237, 'mase': 4.660649316741644, 'smape': 0.26399661410265723, 'mdrae


__Russia: Forecasts Evaluation: FEWNet and baselines:24M (table 06)__

In [13]:
actual = np.array([3.046190,2.423893,2.311721,2.546287,3.098436,3.026511,3.211885,3.366394,3.573624,3.666989,3.977587,
4.423442,4.912471,5.194607,5.666425,5.782023,5.519744,6.014352,6.511526,6.469069,6.692744,7.408086,
8.135334,8.403766])
rw = np.array([3.55,
3.56,
3.57,
3.59,
3.60,
3.61,
3.62,
3.63,
3.64,
3.65,
3.67,
3.68,
3.69,
3.70,
3.71,
3.72,
3.74,
3.75,
3.76,
3.77,
3.78,
3.79,
3.80,
3.82])
rwd = np.array([3.50,
3.46,
3.42,
3.38,
3.34,
3.29,
3.25,
3.21,
3.17,
3.13,
3.09,
3.05,
3.01,
2.97,
2.93,
2.89,
2.85,
2.81,
2.77,
2.72,
2.68,
2.64,
2.60,
2.56])
ar = np.array([3.48,
3.56,
3.72,
3.91,
4.12,
4.33,
4.53,
4.72,
4.89,
5.06,
5.22,
5.37,
5.51,
5.64,
5.77,
5.89,
6.00,
6.10,
6.20,
6.30,
6.39,
6.47,
6.55,
6.62])
msgarch = np.array([3.72,
4.46,
5.10,
5.70,
5.70,
6.02,
5.61,
5.99,
6.09,
6.22,
5.43,
5.73,
5.71,
5.95,
6.13,
6.90,
6.86,
6.43,
5.86,
6.16,
6.66,
6.84,
7.52,
5.93])
deepar = np.array([7.10,
7.58,
7.71,
7.94,
7.95,
7.70,
7.53,
7.36,
7.00,
6.68,
6.69,
6.05,
6.16,
6.28,
6.22,
5.74,
5.76,
5.49,
5.46,
5.10,
4.88,
4.86,
4.68,
4.60])
arnnx = np.array([10.11,
10.02,
9.76,
9.29,
9.64,
8.76,
9.11,
9.77,
9.50,
9.63,
10.81,
11.88,
9.29,
10.26,
11.05,
10.97,
11.28,
11.31,
12.59,
12.44,
13.36,
14.45,
12.94,
12.06])
nbeats = np.array([4.61,
4.30,
4.97,
6.09,
5.84,
5.21,
5.80,
6.05,
7.99,
8.80,
8.40,
9.54,
11.13,
11.27,
11.30,
11.59,
12.77,
13.11,
15.19,
15.16,
13.85,
13.57,
13.79,
10.78])
arfimax = np.array([17.05,
16.54,
16.04,
15.63,
15.36,
15.27,
15.31,
15.45,
15.62,
15.78,
15.93,
16.12,
16.42,
16.82,
17.27,
17.63,
17.81,
17.75,
17.50,
17.14,
16.73,
16.34,
15.98,
15.69])
sarimax = np.array([14.81,
15.27,
15.75,
16.11,
16.19,
16.01,
15.70,
15.20,
14.84,
14.50,
14.18,
14.02,
14.08,
14.10,
13.88,
13.57,
13.19,
13.03,
12.85,
13.17,
13.50,
13.44,
13.06,
12.56])
arimax = np.array([14.92,
15.38,
15.80,
16.04,
16.03,
15.77,
15.34,
14.84,
14.33,
13.89,
13.58,
13.41,
13.34,
13.26,
13.05,
12.72,
12.35,
12.13,
12.17,
12.42,
12.70,
12.77,
12.54,
12.08])
lr = 	np.array([4.50,
10.31,
11.89,
11.49,
11.24,
11.14,
11.18,
11.33,
11.49,
11.64,
11.73,
11.78,
11.91,
12.11,
12.43,
12.73,
12.93,
12.92,
12.73,
12.42,
12.04,
11.67,
11.32,
11.05])
ardl = np.array([7.61,
10.01,
9.27,
8.71,
9.52,
10.29,
10.50,
10.70,
11.15,
11.54,
11.58,
11.62,
12.02,
12.62,
13.09,
13.38,
13.45,
13.27,
12.91,
12.38,
11.90,
11.61,
11.34,
11.09])
xgboost = np.array([3.74,
3.59,
3.59,
3.59,
3.59,
3.59,
3.59,
3.59,
3.59,
3.59,
3.59,
3.59,
3.59,
3.59,
3.66,
3.66,
3.66,
3.66,
3.66,
3.66,
3.64,
3.59,
3.59,
3.59])
tft = 	np.array([6.13,
7.10,
6.47,
6.15,
7.08,
6.61,
7.50,
6.86,
6.16,
6.98,
6.55,
6.55,
6.50,
6.58,
5.81,
6.36,
6.28,
6.29,
6.40,
5.59,
6.26,
5.46,
4.99,
5.74])
warimax = np.array([10.43,
9.86,
9.32,
9.02,
8.68,
8.57,
8.65,
8.72,
9.00,
9.17,
9.42,
9.80,
10.13,
10.60,
10.91,
11.08,
11.09,
10.83,
10.59,
10.26,
9.94,
9.70,
9.37,
9.13])
fewnet = np.array([4.34,
4.45,
4.25,
3.82,
3.36,
3.17,
3.28,
3.02,
3.05,
3.39,
4.00,
4.67,
5.47,
6.19,
7.03,
7.89,
8.85,
9.52,
9.89,
10.44,
10.76,
10.97,
11.31,
11.73])

In [14]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results


print("Evaluation_metrics:  rw - ", evaluate(actual,rw))
print("Measuring Theils U1: rw - ", theils_u1(actual,rw))

print("Evaluation_metrics:  rwd - ", evaluate(actual,rwd))
print("Measuring Theils U1: rwd - ", theils_u1(actual,rwd))

print("Evaluation_metrics:  ar - ", evaluate(actual,ar))
print("Measuring Theils U1: ar - ", theils_u1(actual,ar))

print("Evaluation_metrics:  msgarch - ", evaluate(actual,msgarch))
print("Measuring Theils U1: msgarch - ", theils_u1(actual,msgarch))

print("Evaluation_metrics:  deepar - ", evaluate(actual,deepar))
print("Measuring Theils U1: deepar - ", theils_u1(actual,deepar))

print("Evaluation_metrics:  arnnx - ", evaluate(actual,arnnx))
print("Measuring Theils U1: arnnx - ", theils_u1(actual,arnnx))

print("Evaluation_metrics:  nbeats - ", evaluate(actual,nbeats))
print("Measuring Theils U1: nbeats - ", theils_u1(actual,nbeats))

print("Evaluation_metrics:  arfimax - ", evaluate(actual,arfimax))
print("Measuring Theils U1: arfimax - ", theils_u1(actual,arfimax))

print("Evaluation_metrics:  sarimax - ", evaluate(actual,sarimax))
print("Measuring Theils U1: sarimax - ", theils_u1(actual,sarimax))

print("Evaluation_metrics:  arimax - ", evaluate(actual,arimax))
print("Measuring Theils U1: arimax - ", theils_u1(actual,arimax))

print("Evaluation_metrics:  lr - ", evaluate(actual,lr))
print("Measuring Theils U1: lr - ", theils_u1(actual,lr))

print("Evaluation_metrics:  ardl - ", evaluate(actual,ardl))
print("Measuring Theils U1: ardl - ", theils_u1(actual,ardl))

print("Evaluation_metrics:  xgboost - ", evaluate(actual,xgboost))
print("Measuring Theils U1: xgboost - ", theils_u1(actual,xgboost))

print("Evaluation_metrics:  tft - ", evaluate(actual,tft))
print("Measuring Theils U1: tft - ", theils_u1(actual,tft))

print("Evaluation_metrics:  warimax - ", evaluate(actual,warimax))
print("Measuring Theils U1: warimax - ", theils_u1(actual,warimax))

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

Evaluation_metrics:  rw -  {'rmse': 2.061920369021235, 'mase': 4.869301437832342, 'smape': 0.34039984258644984, 'mdrae': 4.578073948545504, 'mdape': 0.33384926992624275}
Measuring Theils U1: rw -  0.2337792157026489
Evaluation_metrics:  rwd -  {'rmse': 2.738447313689152, 'mase': 6.399132404271898, 'smape': 0.4787911288049116, 'mdrae': 5.799746935089433, 'mdape': 0.4278544681534838}
Measuring Theils U1: rwd -  0.3348185766456566
Evaluation_metrics:  ar -  {'rmse': 1.02784796858193, 'mase': 2.673775483620843, 'smape': 0.20207957855357686, 'mdrae': 2.1230175725015976, 'mdape': 0.20356405640550818}
Measuring Theils U1: ar -  0.09795032946392844
Evaluation_metrics:  msgarch -  {'rmse': 1.8181102374951488, 'mase': 4.632206158550134, 'smape': 0.323153286287231, 'mdrae': 4.711706440705008, 'mdape': 0.2685876200005698}
Measuring Theils U1: msgarch -  0.1633113888671574
Evaluation_metrics:  deepar -  {'rmse': 3.256978451574097, 'mase': 8.388846638634401, 'smape': 0.5082424496080964, 'mdrae': 8.2

__India: Forecasts Evaluation: FEWNet and baselines:12M (table 05)__

In [15]:
actual = np.array([3.686636 ,3.162966 ,4.494493 ,5.665658 ,5.139860 ,
                   5.257646 ,5.577841 ,5.263880 ,4.811442 ,4.403048
                   ,4.518828 ,4.837364])
rw = np.array([5.31,
5.33,
5.36,
5.39,
5.41,
5.44,
5.46,
5.49,
5.51,
5.54,
5.56,
5.59])
rwd = np.array([5.29,
5.29,
5.30,
5.30,
5.31,
5.31,
5.32,
5.32,
5.33,
5.33,
5.34,
5.34])
ar = np.array([5.27,
5.37,
5.48,
5.58,
5.68,
5.77,
5.86,
5.94,
6.02,
6.09,
6.15,
6.21])
msgarch = np.array([5.20,
5.81,
6.02,
6.17,
6.51,
7.12,
5.90,
7.03,
7.03,
8.48,
8.27,
5.74])
deepar = np.array([6.24,
7.12,
7.09,
7.12,
7.53,
8.00,
8.03,
8.72,
8.97,
9.36,
9.52,
9.52])
arnnx = np.array([3.44,
2.83,
3.03,
3.12,
2.85,
2.66,
2.77,
3.29,
3.88,
4.25,
4.28,
4.08])
nbeats = np.array([4.79,
3.70,
5.14,
5.80,
6.03,
5.36,
5.12,
3.89,
3.74,
3.39,
2.74,
2.82])
arfimax = np.array([6.01,
6.29,
6.14,
5.66,
5.19,
5.03,
5.35,
6.03,
6.74,
7.17,
7.19,
6.97])
sarimax = np.array([2.79,
2.85,
2.26,
1.53,
1.32,
1.91,
3.05,
3.42,
3.87,
3.87,
3.63,
3.59])
arimax = np.array([2.54,
2.18,
2.22,
2.13,
1.71,
1.55,
2.24,
3.47,
4.28,
4.14,
3.49,
3.19])
lr = 	np.array([3.40,
2.44,
2.32,
2.03,
1.78,
1.63,
1.89,
2.33,
2.88,
3.39,
3.48,
3.64])
ardl = np.array([3.36,
2.53,
2.71,
2.88,
2.62,
2.29,
2.43,
3.10,
3.93,
4.44,
4.43,
4.08])
xgboost = np.array([3.95,
3.51,
3.35,
3.65,
3.07,
3.56,
3.07,
3.19,
3.06,
2.81,
2.88,
3.17])
tft = 	np.array([6.18,
7.53,
5.06,
3.97,
5.09,
4.46,
6.41,
4.85,
3.97,
5.95,
5.14,
4.72])
warimax = np.array([4.01,
4.35,
4.35,
3.95,
3.52,
3.41,
3.74,
4.35,
4.99,
5.41,
5.49,
5.33])
fewnet = np.array([4.37,
5.04,
5.83,
6.55,
6.04,
5.41,
4.80,
4.40,
4.26,
4.56,
4.57,
5.22])

In [16]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results


print("Evaluation_metrics:  rw - ", evaluate(actual,rw))
print("Measuring Theils U1: rw - ", theils_u1(actual,rw))

print("Evaluation_metrics:  rwd - ", evaluate(actual,rwd))
print("Measuring Theils U1: rwd - ", theils_u1(actual,rwd))

print("Evaluation_metrics:  ar - ", evaluate(actual,ar))
print("Measuring Theils U1: ar - ", theils_u1(actual,ar))

print("Evaluation_metrics:  msgarch - ", evaluate(actual,msgarch))
print("Measuring Theils U1: msgarch - ", theils_u1(actual,msgarch))

print("Evaluation_metrics:  deepar - ", evaluate(actual,deepar))
print("Measuring Theils U1: deepar - ", theils_u1(actual,deepar))

print("Evaluation_metrics:  arnnx - ", evaluate(actual,arnnx))
print("Measuring Theils U1: arnnx - ", theils_u1(actual,arnnx))

print("Evaluation_metrics:  nbeats - ", evaluate(actual,nbeats))
print("Measuring Theils U1: nbeats - ", theils_u1(actual,nbeats))

print("Evaluation_metrics:  arfimax - ", evaluate(actual,arfimax))
print("Measuring Theils U1: arfimax - ", theils_u1(actual,arfimax))

print("Evaluation_metrics:  sarimax - ", evaluate(actual,sarimax))
print("Measuring Theils U1: sarimax - ", theils_u1(actual,sarimax))

print("Evaluation_metrics:  arimax - ", evaluate(actual,arimax))
print("Measuring Theils U1: arimax - ", theils_u1(actual,arimax))

print("Evaluation_metrics:  lr - ", evaluate(actual,lr))
print("Measuring Theils U1: lr - ", theils_u1(actual,lr))

print("Evaluation_metrics:  ardl - ", evaluate(actual,ardl))
print("Measuring Theils U1: ardl - ", theils_u1(actual,ardl))

print("Evaluation_metrics:  xgboost - ", evaluate(actual,xgboost))
print("Measuring Theils U1: xgboost - ", theils_u1(actual,xgboost))

print("Evaluation_metrics:  tft - ", evaluate(actual,tft))
print("Measuring Theils U1: tft - ", theils_u1(actual,tft))

print("Evaluation_metrics:  warimax - ", evaluate(actual,warimax))
print("Measuring Theils U1: warimax - ", theils_u1(actual,warimax))

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

Evaluation_metrics:  rw -  {'rmse': 0.9899000497529198, 'mase': 1.5319119524936373, 'smape': 0.16337058621955172, 'mdrae': 1.5439861376683615, 'mdape': 0.15038743548283953}
Measuring Theils U1: rw -  0.09668713391391495
Evaluation_metrics:  rwd -  {'rmse': 0.9144226713255563, 'mase': 1.3436426902412522, 'smape': 0.14584704843229593, 'mdrae': 0.8052624179624096, 'mdape': 0.10584150475303086}
Measuring Theils U1: rwd -  0.09050704697613471
Evaluation_metrics:  ar -  {'rmse': 1.2348712126510062, 'mase': 2.090878153324105, 'smape': 0.21212255475289502, 'mdrae': 2.671212409804483, 'mdape': 0.2352270248009155}
Measuring Theils U1: ar -  0.11670597974970655
Evaluation_metrics:  msgarch -  {'rmse': 2.1770925700348234, 'mase': 3.6770358827819196, 'smape': 0.32921003593555415, 'mdrae': 4.903562478152491, 'mdape': 0.34681757250241785}
Measuring Theils U1: msgarch -  0.18987551214088952
Evaluation_metrics:  deepar -  {'rmse': 3.5475866912359124, 'mase': 6.6140363739191255, 'smape': 0.5244980565232

__India: Forecasts Evaluation: FEWNet and baselines:24M (table 06)__

In [19]:
actual = np.array([9.634551,7.491857,6.840391,5.501618,5.448718,5.095541,5.063291,5.329154,5.625000,5.636574,5.902162,
5.284787,3.686636,3.162966,4.494493,5.665658,5.139860,5.257646,5.577841,5.263880,4.811442,4.403048,
4.518828,4.837364])
rw = np.array([8.63,
8.65,
8.67,
8.70,
8.72,
8.74,
8.76,
8.78,
8.81,
8.83,
8.85,
8.87,
8.89,
8.91,
8.94,
8.96,
8.98,
9.00,
9.02,
9.05,
9.07,
9.09,
9.11,
9.13])
rwd = np.array([8.63,
8.65,
8.68,
8.70,
8.72,
8.74,
8.77,
8.79,
8.81,
8.83,
8.86,
8.88,
8.90,
8.92,
8.95,
8.97,
8.99,
9.01,
9.04,
9.06,
9.08,
9.10,
9.13,
9.15])
ar = np.array([8.72,
8.68,
8.61,
8.53,
8.46,
8.40,
8.34,
8.28,
8.23,
8.18,
8.13,
8.08,
8.04,
8.00,
7.97,
7.93,
7.90,
7.87,
7.84,
7.82,
7.79,
7.77,
7.75,
7.73])
msgarch = np.array([8.07,
7.62,
7.81,
8.11,
7.94,
8.39,
7.21,
7.91,
7.11,
6.32,
6.00,
6.05,
6.35,
5.71,
5.52,
5.83,
6.17,
6.24,
5.40,
6.06,
6.73,
7.76,
8.08,
6.02])
deepar = np.array([5.99,
7.04,
6.96,
6.53,
6.85,
7.39,
7.34,
7.89,
8.14,
8.13,
7.71,
7.29,
6.55,
6.64,
6.06,
6.14,
5.88,
5.63,
5.36,
5.18,
5.17,
5.75,
6.00,
6.31])
arnnx = np.array([3.89,
1.69,
3.11,
3.90,
3.33,
2.83,
2.70,
3.34,
4.28,
4.62,
4.45,
4.10,
3.85,
3.87,
4.17,
4.53,
4.59,
4.38,
4.20,
3.87,
3.69,
4.11,
5.06,
5.68])
nbeats = np.array([7.84,
6.51,
6.18,
6.71,
6.34,
6.85,
7.22,
6.88,
6.84,
7.61,
6.50,
5.94,
6.91,
8.21,
8.92,
8.83,
8.41,
9.29,
9.52,
11.93,
13.14,
11.86,
12.40,
12.39])
arfimax = np.array([7.00,
6.93,
6.69,
6.36,
6.06,
5.97,
6.25,
6.86,
7.54,
7.99,
8.06,
7.86,
7.66,
7.69,
7.97,
8.29,
8.39,
8.16,
7.75,
7.49,
7.63,
8.17,
8.84,
9.29])
sarimax = np.array([2.97,
3.72,
3.86,
3.87,
3.58,
3.08,
2.79,
3.22,
3.64,
4.19,
4.45,
4.61,
4.35,
4.77,
4.30,
4.53,
5.33,
6.30,
6.86,
6.22,
6.02,
5.90,
5.87,
6.43])
arimax = np.array([3.76,
3.69,
3.76,
4.01,
4.06,
3.66,
3.15,
3.13,
3.77,
4.65,
5.15,
5.08,
4.71,
4.46,
4.60,
5.12,
5.82,
6.43,
6.73,
6.63,
6.30,
6.09,
6.24,
6.68])
lr = 	np.array([5.99,
3.42,
3.16,
2.91,
2.36,
2.06,
2.06,
2.56,
3.07,
3.54,
3.44,
3.06,
3.23,
3.58,
3.80,
4.04,
4.02,
3.57,
2.89,
2.68,
2.91,
3.43,
4.16,
4.75])
ardl = np.array([1.30,
1.52,
4.12,
4.40,
2.60,
2.11,
3.45,
4.81,
5.08,
4.65,
4.25,
4.09,
4.09,
4.27,
4.53,
4.62,
4.38,
3.86,
3.43,
3.42,
3.86,
4.51,
5.02,
5.14])
xgboost = np.array([4.42,
3.98,
3.65,
2.93,
3.04,
3.26,
3.26,
3.97,
4.22,
4.24,
4.24,
4.22,
4.22,
4.21,
4.18,
4.22,
4.22,
4.14,
4.21,
3.54,
3.36,
4.19,
5.09,
5.31])
tft = 	np.array([9.51,
11.22,
8.87,
6.66,
9.72,
8.86,
10.55,
8.33,
6.90,
9.95,
9.33,
9.33,
10.37,
11.40,
9.27,
11.94,
12.22,
13.43,
15.13,
13.76,
15.58,
13.32,
10.10,
14.45])
warimax = np.array([3.67,
3.47,
3.46,
3.06,
2.68,
2.91,
3.40,
4.30,
5.24,
5.78,
6.05,
5.92,
5.83,
6.13,
6.52,
6.98,
7.08,
6.69,
6.25,
5.91,
6.01,
6.62,
7.20,
7.58])
fewnet = np.array([7.69,
6.34,
5.92,
5.11,
4.53,
4.15,
3.95,
5.57,
5.37,
4.25,
3.85,
3.37,
3.86,
4.75,
5.39,
5.65,
4.78,
4.36,
4.87,
4.06,
4.28,
4.73,
5.04,
4.72])

In [20]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results


print("Evaluation_metrics:  rw - ", evaluate(actual,rw))
print("Measuring Theils U1: rw - ", theils_u1(actual,rw))

print("Evaluation_metrics:  rwd - ", evaluate(actual,rwd))
print("Measuring Theils U1: rwd - ", theils_u1(actual,rwd))

print("Evaluation_metrics:  ar - ", evaluate(actual,ar))
print("Measuring Theils U1: ar - ", theils_u1(actual,ar))

print("Evaluation_metrics:  msgarch - ", evaluate(actual,msgarch))
print("Measuring Theils U1: msgarch - ", theils_u1(actual,msgarch))

print("Evaluation_metrics:  deepar - ", evaluate(actual,deepar))
print("Measuring Theils U1: deepar - ", theils_u1(actual,deepar))

print("Evaluation_metrics:  arnnx - ", evaluate(actual,arnnx))
print("Measuring Theils U1: arnnx - ", theils_u1(actual,arnnx))

print("Evaluation_metrics:  nbeats - ", evaluate(actual,nbeats))
print("Measuring Theils U1: nbeats - ", theils_u1(actual,nbeats))

print("Evaluation_metrics:  arfimax - ", evaluate(actual,arfimax))
print("Measuring Theils U1: arfimax - ", theils_u1(actual,arfimax))

print("Evaluation_metrics:  sarimax - ", evaluate(actual,sarimax))
print("Measuring Theils U1: sarimax - ", theils_u1(actual,sarimax))

print("Evaluation_metrics:  arimax - ", evaluate(actual,arimax))
print("Measuring Theils U1: arimax - ", theils_u1(actual,arimax))

print("Evaluation_metrics:  lr - ", evaluate(actual,lr))
print("Measuring Theils U1: lr - ", theils_u1(actual,lr))

print("Evaluation_metrics:  ardl - ", evaluate(actual,ardl))
print("Measuring Theils U1: ardl - ", theils_u1(actual,ardl))

print("Evaluation_metrics:  xgboost - ", evaluate(actual,xgboost))
print("Measuring Theils U1: xgboost - ", theils_u1(actual,xgboost))

print("Evaluation_metrics:  tft - ", evaluate(actual,tft))
print("Measuring Theils U1: tft - ", theils_u1(actual,tft))

print("Evaluation_metrics:  warimax - ", evaluate(actual,warimax))
print("Measuring Theils U1: warimax - ", theils_u1(actual,warimax))

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

Evaluation_metrics:  rw -  {'rmse': 3.7228611467557005, 'mase': 6.195391103065349, 'smape': 0.512477209686288, 'mdrae': 10.765736223992977, 'mdape': 0.6950976231695414}
Measuring Theils U1: rw -  0.2580927628929062
Evaluation_metrics:  rwd -  {'rmse': 3.731725546092116, 'mase': 6.20988395860427, 'smape': 0.5133242321008957, 'mdrae': 10.812657908208227, 'mdape': 0.6969947310341758}
Measuring Theils U1: rwd -  0.25855647575911767
Evaluation_metrics:  ar -  {'rmse': 2.9203220415489284, 'mase': 4.870019464031013, 'smape': 0.4289481159243113, 'mdrae': 8.244371858608192, 'mdape': 0.5437299101788028}
Measuring Theils U1: ar -  0.21356142096677064
Evaluation_metrics:  msgarch -  {'rmse': 1.9168800646336308, 'mase': 2.769691795690258, 'smape': 0.2641945464590842, 'mdrae': 3.7127232075141507, 'mdape': 0.23632455238496608}
Measuring Theils U1: msgarch -  0.15385765549294597
Evaluation_metrics:  deepar -  {'rmse': 1.8638063102666633, 'mase': 2.685015403132891, 'smape': 0.2569428825540427, 'mdrae':

__China: Forecasts Evaluation: FEWNet and baselines:12M (table 05)__

In [21]:
actual = np.array([0.2697842 ,-0.2216312 ,-0.4227999  ,0.3219466  ,0.9226485  ,
                   1.6443494  ,1.2401969  ,1.9125597  ,0.6415800  ,0.4613803  ,
                   1.4328405  ,2.4772978])
rw = np.array([-0.45,
-0.45,
-0.45,
-0.45,
-0.45,
-0.45,
-0.45,
-0.45,
-0.45,
-0.45,
-0.45,
-0.45])
rwd = np.array([-0.47,
-0.49,
-0.50,
-0.52,
-0.54,
-0.56,
-0.58,
-0.59,
-0.61,
-0.63,
-0.65,
-0.67])
ar = np.array([-0.61,
-0.64,
-0.47,
-0.26,
-0.02,
0.22,
0.45,
0.66,
0.86,
1.03,
1.19,
1.33])
msgarch = np.array([1.15,
1.67,
1.98,
2.16,
2.37,
2.41,
2.48,
2.93,
2.83,
3.39,
3.56,
2.85])
deepar = np.array([1.90,
1.58,
1.68,
1.69,
1.55,
1.56,
2.19,
1.85,
2.28,
2.38,
2.39,
1.85])
arnnx = np.array([0.64,
1.40,
1.88,
2.20,
2.52,
2.89,
3.31,
3.66,
3.82,
3.73,
3.46,
3.12])
nbeats = np.array([0.10,
0.20,
-0.02,
-0.14,
0.15,
0.32,
0.34,
1.15,
1.40,
2.22,
2.73,
2.82])
arfimax = np.array([10.43,
10.49,
10.49,
10.60,
11.03,
11.69,
12.33,
12.63,
12.44,
11.86,
11.12,
10.50])
sarimax = np.array([4.74,
3.92,
4.68,
4.73,
5.75,
6.42,
6.66,
6.80,
6.01,
5.15,
4.72,
4.30])
arimax = np.array([5.26,
5.09,
5.21,
5.68,
6.45,
7.18,
7.53,
7.32,
6.64,
5.78,
5.05,
4.61])
lr = 	np.array([0.44,
1.60,
2.24,
2.54,
2.97,
3.60,
4.08,
4.39,
4.31,
3.70,
3.45,
3.21])
ardl = np.array([3.93,
2.99,
2.76,
3.95,
4.50,
5.20,
5.67,
5.39,
4.67,
3.81,
3.05,
2.63])
xgboost = np.array([-0.86,
-1.23,
-0.69,
-0.43,
-0.21,
0.66,
1.56,
1.81,
2.23,
2.29,
2.25,
2.42])
tft = 	np.array([-0.87,
-0.62,
-2.07,
-2.62,
-1.56,
-1.93,
-0.74,
-1.24,
-1.51,
0.34,
0.60,
0.87])
warimax = np.array([4.48,
4.85,
4.93,
5.00,
5.42,
6.14,
6.65,
6.88,
6.77,
6.18,
5.57,
5.06])
fewnet = np.array([-0.19,
-0.45,
-0.17,
0.50,
1.24,
2.03,
2.44,
3.15,
2.86,
3.34,
4.45,
5.44])

In [22]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results


print("Evaluation_metrics:  rw - ", evaluate(actual,rw))
print("Measuring Theils U1: rw - ", theils_u1(actual,rw))

print("Evaluation_metrics:  rwd - ", evaluate(actual,rwd))
print("Measuring Theils U1: rwd - ", theils_u1(actual,rwd))

print("Evaluation_metrics:  ar - ", evaluate(actual,ar))
print("Measuring Theils U1: ar - ", theils_u1(actual,ar))

print("Evaluation_metrics:  msgarch - ", evaluate(actual,msgarch))
print("Measuring Theils U1: msgarch - ", theils_u1(actual,msgarch))

print("Evaluation_metrics:  deepar - ", evaluate(actual,deepar))
print("Measuring Theils U1: deepar - ", theils_u1(actual,deepar))

print("Evaluation_metrics:  arnnx - ", evaluate(actual,arnnx))
print("Measuring Theils U1: arnnx - ", theils_u1(actual,arnnx))

print("Evaluation_metrics:  nbeats - ", evaluate(actual,nbeats))
print("Measuring Theils U1: nbeats - ", theils_u1(actual,nbeats))

print("Evaluation_metrics:  arfimax - ", evaluate(actual,arfimax))
print("Measuring Theils U1: arfimax - ", theils_u1(actual,arfimax))

print("Evaluation_metrics:  sarimax - ", evaluate(actual,sarimax))
print("Measuring Theils U1: sarimax - ", theils_u1(actual,sarimax))

print("Evaluation_metrics:  arimax - ", evaluate(actual,arimax))
print("Measuring Theils U1: arimax - ", theils_u1(actual,arimax))

print("Evaluation_metrics:  lr - ", evaluate(actual,lr))
print("Measuring Theils U1: lr - ", theils_u1(actual,lr))

print("Evaluation_metrics:  ardl - ", evaluate(actual,ardl))
print("Measuring Theils U1: ardl - ", theils_u1(actual,ardl))

print("Evaluation_metrics:  xgboost - ", evaluate(actual,xgboost))
print("Measuring Theils U1: xgboost - ", theils_u1(actual,xgboost))

print("Evaluation_metrics:  tft - ", evaluate(actual,tft))
print("Measuring Theils U1: tft - ", theils_u1(actual,tft))

print("Evaluation_metrics:  warimax - ", evaluate(actual,warimax))
print("Measuring Theils U1: warimax - ", theils_u1(actual,warimax))

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

Evaluation_metrics:  rw -  {'rmse': 1.5814582384278058, 'mase': 2.018272292450006, 'smape': 1.7285308862693662, 'mdrae': 2.2850743434830028, 'mdape': 1.3384535418132404}
Measuring Theils U1: rw -  0.9448743673739093
Evaluation_metrics:  rwd -  {'rmse': 1.7056237024593608, 'mase': 2.195245979687263, 'smape': 1.7434627263299314, 'mdrae': 2.434899073494707, 'mdape': 1.4606560130320991}
Measuring Theils U1: rwd -  0.95039302680613
Evaluation_metrics:  ar -  {'rmse': 0.8239597556259518, 'mase': 1.0686489832824013, 'smape': 1.0295806854641663, 'mdrae': 1.0984630964713957, 'mdape': 0.7605606081307084}
Measuring Theils U1: ar -  0.4175516727206946
Evaluation_metrics:  msgarch -  {'rmse': 1.7509042835226158, 'mase': 2.3972839607462455, 'smape': 1.069851641958242, 'mdrae': 2.4094338635503845, 'mdape': 2.415679213800885}
Measuring Theils U1: msgarch -  0.46173190285295046
Evaluation_metrics:  deepar -  {'rmse': 1.328492846508596, 'mase': 1.7280992312162613, 'smape': 0.9393726918592739, 'mdrae': 1

__China: Forecasts Evaluation: FEWNet and baselines:24M (table 06)__

In [23]:
actual = np.array([4.4131455,5.4205607,5.1803885,4.2711235,3.2467532,2.4118738,2.5069638,1.7576318,2.3875115,
1.7304189,0.5415162,-0.4496403,0.2697842,-0.2216312,-0.4227999,0.3219466,0.9226485,1.6443494,
1.2401969,1.9125597,0.6415800,0.4613803,1.4328405,2.4772978])
rw = np.array([4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51,
4.51])
rwd = np.array([4.52,
4.53,
4.53,
4.54,
4.55,
4.55,
4.56,
4.57,
4.57,
4.58,
4.59,
4.59,
4.60,
4.61,
4.61,
4.62,
4.63,
4.63,
4.64,
4.65,
4.66,
4.66,
4.67,
4.68])
ar = np.array([4.61,
4.67,
4.57,
4.46,
4.32,
4.19,
4.06,
3.94,
3.83,
3.73,
3.64,
3.56,
3.48,
3.42,
3.36,
3.30,
3.25,
3.21,
3.17,
3.13,
3.10,
3.07,
3.04,
3.02])
msgarch = np.array([4.10,
3.78,
3.87,
3.90,
3.86,
4.38,
3.71,
3.47,
3.48,
3.32,
2.78,
2.73,
3.10,
2.87,
2.78,
2.91,
3.11,
2.87,
3.25,
3.12,
3.68,
3.54,
3.81,
3.35])
deepar = np.array([1.94,
1.63,
1.88,
1.79,
1.68,
1.49,
1.99,
1.84,
2.20,
2.41,
2.25,
1.79,
1.74,
2.08,
2.01,
2.04,
2.01,
2.03,
1.71,
1.49,
1.60,
1.26,
1.26,
1.38])
arnnx = np.array([2.14,
3.08,
3.58,
4.02,
3.73,
4.22,
5.17,
5.08,
4.37,
3.61,
3.17,
2.10,
3.10,
2.31,
2.23,
2.07,
1.72,
1.76,
1.29,
1.36,
1.22,
0.94,
1.19,
2.20])
nbeats = np.array([3.76,
4.09,
3.80,
4.56,
4.12,
4.41,
3.73,
4.14,
3.41,
2.63,
2.44,
2.18,
2.02,
1.94,
2.04,
1.18,
1.47,
0.54,
0.95,
0.55,
1.10,
1.73,
1.69,
1.76])
arfimax = np.array([19.84,
19.91,
19.90,
20.09,
20.59,
21.34,
22.04,
22.32,
22.08,
21.49,
20.76,
20.16,
19.83,
19.74,
19.74,
19.70,
19.55,
19.32,
19.08,
18.91,
18.88,
19.01,
19.24,
19.47])
sarimax = np.array([4.50,
4.95,
4.30,
4.23,
4.18,
4.06,
3.75,
3.40,
3.85,
4.09,
4.62,
4.64,
4.67,
4.08,
5.26,
4.94,
5.15,
5.22,
5.43,
5.61,
5.40,
5.33,
5.74,
6.10])
arimax = np.array([5.41,
5.38,
5.34,
5.18,
4.93,
4.66,
4.45,
4.41,
4.55,
4.83,
5.12,
5.32,
5.39,
5.37,
5.40,
5.56,
5.82,
6.04,
6.12,
6.03,
5.92,
5.98,
6.28,
6.73])
lr = 	np.array([3.94,
3.59,
3.22,
2.92,
3.15,
3.50,
3.82,
3.90,
3.75,
3.42,
3.03,
2.67,
2.44,
2.28,
2.10,
1.95,
1.84,
1.69,
1.53,
1.42,
1.43,
1.55,
1.72,
1.90])
ardl = np.array([3.83,
3.64,
3.62,
3.91,
4.64,
5.35,
5.68,
5.46,
4.75,
3.88,
3.13,
2.70,
2.58,
2.56,
2.51,
2.19,
1.84,
1.52,
1.23,
1.09,
1.10,
1.26,
1.48,
1.60])
xgboost = np.array([-0.97,
-0.98,
-0.69,
-0.55,
-0.46,
0.58,
1.21,
1.84,
2.05,
2.06,
2.30,
2.45,
2.50,
2.80,
2.89,
2.52,
2.44,
2.44,
2.32,
2.13,
2.11,
2.00,
1.87,
1.72])
tft = 	np.array([2.11,
2.61,
0.84,
-0.20,
1.29,
0.81,
2.21,
1.23,
0.39,
2.28,
1.70,
1.64,
2.03,
2.47,
1.42,
2.48,
2.31,
2.47,
3.01,
2.23,
2.98,
2.08,
1.63,
2.19])
warimax = np.array([8.96,
9.21,
9.18,
9.39,
9.92,
10.43,
10.89,
10.99,
10.73,
10.21,
9.44,
8.86,
8.62,
8.58,
8.53,
8.42,
8.36,
8.19,
7.94,
7.83,
7.94,
8.11,
8.28,
8.53])
fewnet = np.array([4.10,
3.71,
3.68,
3.25,
3.28,
3.31,
3.58,
3.63,
3.40,
3.39,
3.24,
2.90,
3.20,
3.75,
3.38,
3.42,
3.06,
2.82,
2.58,
2.39,
2.44,
2.16,
1.88,
2.08])

In [24]:
# Generate the values of the different evaluation metrics
def evaluate(actual: np.ndarray, predicted: np.ndarray, metrics=('rmse',
                                                               'mase',
                                                               'smape',
                                                               'mdrae',
                                                               'mdape'
                                                              )):
 results = {}
 for name in metrics:
     try:
         results[name] = METRICS[name](actual, predicted)
     except Exception as err:
         results[name] = np.nan
         print('Unable to compute metric {0}: {1}'.format(name, err))
 return results


print("Evaluation_metrics:  rw - ", evaluate(actual,rw))
print("Measuring Theils U1: rw - ", theils_u1(actual,rw))

print("Evaluation_metrics:  rwd - ", evaluate(actual,rwd))
print("Measuring Theils U1: rwd - ", theils_u1(actual,rwd))

print("Evaluation_metrics:  ar - ", evaluate(actual,ar))
print("Measuring Theils U1: ar - ", theils_u1(actual,ar))

print("Evaluation_metrics:  msgarch - ", evaluate(actual,msgarch))
print("Measuring Theils U1: msgarch - ", theils_u1(actual,msgarch))

print("Evaluation_metrics:  deepar - ", evaluate(actual,deepar))
print("Measuring Theils U1: deepar - ", theils_u1(actual,deepar))

print("Evaluation_metrics:  arnnx - ", evaluate(actual,arnnx))
print("Measuring Theils U1: arnnx - ", theils_u1(actual,arnnx))

print("Evaluation_metrics:  nbeats - ", evaluate(actual,nbeats))
print("Measuring Theils U1: nbeats - ", theils_u1(actual,nbeats))

print("Evaluation_metrics:  arfimax - ", evaluate(actual,arfimax))
print("Measuring Theils U1: arfimax - ", theils_u1(actual,arfimax))

print("Evaluation_metrics:  sarimax - ", evaluate(actual,sarimax))
print("Measuring Theils U1: sarimax - ", theils_u1(actual,sarimax))

print("Evaluation_metrics:  arimax - ", evaluate(actual,arimax))
print("Measuring Theils U1: arimax - ", theils_u1(actual,arimax))

print("Evaluation_metrics:  lr - ", evaluate(actual,lr))
print("Measuring Theils U1: lr - ", theils_u1(actual,lr))

print("Evaluation_metrics:  ardl - ", evaluate(actual,ardl))
print("Measuring Theils U1: ardl - ", theils_u1(actual,ardl))

print("Evaluation_metrics:  xgboost - ", evaluate(actual,xgboost))
print("Measuring Theils U1: xgboost - ", theils_u1(actual,xgboost))

print("Evaluation_metrics:  tft - ", evaluate(actual,tft))
print("Measuring Theils U1: tft - ", theils_u1(actual,tft))

print("Evaluation_metrics:  warimax - ", evaluate(actual,warimax))
print("Measuring Theils U1: warimax - ", theils_u1(actual,warimax))

print("Evaluation_metrics:  fewnet - ", evaluate(actual,fewnet))
print("Measuring Theils U1: fewnet - ", theils_u1(actual,fewnet))

Evaluation_metrics:  rw -  {'rmse': 3.1439407682643274, 'mase': 3.9448257634708765, 'smape': 1.012927799354132, 'mdrae': 3.863152898425796, 'mdape': 1.6745157091255096}
Measuring Theils U1: rw -  0.4501993507474623
Evaluation_metrics:  rwd -  {'rmse': 3.2344600630972637, 'mase': 4.063222997192853, 'smape': 1.0230097447883648, 'mdrae': 4.0713738172202, 'mdape': 1.7312306208491315}
Measuring Theils U1: rwd -  0.4574141824064194
Evaluation_metrics:  ar -  {'rmse': 2.2342148258440044, 'mase': 2.740339054071565, 'smape': 0.8868119477258164, 'mdrae': 2.6061710518956827, 'mdape': 1.1386035803133057}
Measuring Theils U1: ar -  0.36121548118524366
Evaluation_metrics:  msgarch -  {'rmse': 2.077109928808413, 'mase': 2.634164752194101, 'smape': 0.8912531877882289, 'mdrae': 2.4191127707752473, 'mdape': 0.9464289840211033}
Measuring Theils U1: msgarch -  0.3514165247055698
Evaluation_metrics:  deepar -  {'rmse': 1.7126214094007532, 'mase': 1.9496646614711375, 'smape': 0.8142596730977306, 'mdrae': 1.